In [1]:
import os
import pandas as pd


import glob
import scib
import scanpy as sc
import scvi

from pathlib import Path
from data_manage import *

/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
Global seed set to 0
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [2]:
import scgen

## scvi follow tutorial: https://docs.scvi-tools.org/en/stable/tutorials/notebooks/harmonization.html

In [3]:
ad_file_paths = glob.glob("../../results/simulation/test/derived_data/human_pancreas_norm_complexBatch/*/*h5ad")
raw_ad_file_paths = glob.glob("../../data/raw_data/*h5ad")

In [21]:
adata_raw = sc.read(raw_ad_file_paths[0])
adata_raw

AnnData object with n_obs × n_vars = 16382 × 19093
    obs: 'tech', 'celltype', 'size_factors'
    layers: 'counts'

In [22]:
adata = sc.read(raw_ad_file_paths[0])
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes=2000,
    batch_key = "tech",
    subset = True,
    layer="counts"
)

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  UserWarning,


In [23]:
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="tech")
vae = scvi.model.SCVI(adata, n_layers=2, n_latent=30, gene_likelihood="nb")
vae.train(max_epochs = 1, early_stopping = True)

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/scvi/data/fields/_layer_field.py:79: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  f"{logger_data_loc} does not contain unnormalized count data. "
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Epoch 1/1: 100%|██████████| 1/1 [00:03<00:00,  3.27s/it, loss=2.21e+03, v_num=1]


In [24]:
adata.obsm["X_scVI"] = vae.get_latent_representation()

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
scib.me.pcr_comparison(adata_pre = adata_raw, adata_post = adata, covariate = "celltype")

Variance contribution increased after integration!
Setting PCR comparison score to 0.


0

In [27]:
pcr = scib.me.pcr_comparison(adata_pre = adata_raw, adata_post = adata, covariate = "celltype")
nmi = scib.me.nmi(adata = adata, group1 = "tech",  group2 = "celltype")
ari = scib.me.ari(adata = adata, group1 = "tech",  group2 = "celltype")
print(pcr, nmi, ari)

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Variance contribution increased after integration!
Setting PCR comparison score to 0.
0 0.07837369020085136 0.028379073573485902


In [10]:
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.leiden(adata)

ValueError: Did not find X_scVI in `.obsm.keys()`. You need to compute it first.

## scgen
tutorial: https://scgen.readthedocs.io/en/stable/tutorials/scgen_batch_removal.html

In [7]:
int_metrics = pd.DataFrame(columns = ["ad", "dataset", "c_type", "n_cell", "cell_times", "seq_depth",
                                      "method", "metric", "evaluation", "fig_path", "hyper_para"])
ad_file_paths = glob.glob("../../results/simulation2/test/derived_data/human_pancreas_norm_complexBatch/*/*h5ad") + glob.glob("../../data/derived_data/human_pancreas_norm_complexBatch/*/*h5ad")

In [ ]:
for path in ad_file_paths:
    adata = sc.read(path)
    c_type, n_cell, cell_times, seq_depth = path2para(path)
    # train scgen
    for n_latent in [5, 50, 100]: 
        fig_path = "immune" + c_type + "_" +str(n_cell) + "_" + str(cell_times) + "_" + seq_depth + "_" + str(n_latent)
        model_path ="../../results/scgen_models" + "immune" + c_type + "_" +str(n_cell) + "_" + str(cell_times) + "_" + seq_depth + "_" + str(n_latent)
        
        scgen.SCGEN.setup_anndata(adata, batch_key="tech", labels_key="celltype")
        model = scgen.SCGEN(adata, n_latent = n_latent)
        model.save(model_path + "scgen_batch_removal.pt", overwrite=True, save_anndata = True)
        model.train(
            max_epochs = 1,
            batch_size = 32,
            early_stopping = True,
            early_stopping_patience = 25,
        )

        res = model.batch_removal()



        
        pcr = scib.me.pcr_comparison(adata_pre = adata, adata_post = res, covariate = "celltype")
        nmi = scib.me.nmi(adata = res, group1 = "tech",  group2 = "celltype")
        ari = scib.me.ari(adata = res, group1 = "tech",  group2 = "celltype")
        
        # visualization
        scgen_vis(path, adata, res, fig_path)

        int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                             "scgen", "pcr", pcr, fig_path, n_latent]
        int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                             "scgen", "nmi", nmi, fig_path, n_latent]
        int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", c_type, n_cell, cell_times, seq_depth,
                             "scgen", "ari", ari, fig_path, n_latent]
    print(int_metrics)
    int_metrics.to_csv("scgen_0427.csv")

In [5]:
int_metrics

,ad,dataset,c_type,n_cell,cell_times,seq_depth,method,metric,evaluation
0,path,human_pancreas_norm_complexBatch,alpha,2,0,1000,scgen,pcr,0.000000
1,path,human_pancreas_norm_complexBatch,alpha,2,0,1000,scgen,nmi,0.000000
2,path,human_pancreas_norm_complexBatch,alpha,2,0,1000,scgen,ari,0.000000
3,path,human_pancreas_norm_complexBatch,delta,2,0,5000,scgen,pcr,0.000000
4,path,human_pancreas_norm_complexBatch,delta,2,0,5000,scgen,nmi,0.000000
5,path,human_pancreas_norm_complexBatch,delta,2,0,5000,scgen,ari,0.000000
6,path,human_pancreas_norm_complexBatch,activated_stellate,5,0,10000,scgen,pcr,0.856388
7,path,human_pancreas_norm_complexBatch,activated_stellate,5,0,10000,scgen,nmi,0.000000
8,path,human_pancreas_norm_complexBatch,activated_stellate,5,0,10000,scgen,ari,0.000000
9,path,human_pancreas_norm_complexBatch,activated_stellate,0,0,5000,scgen,pcr,0.000000


In [6]:
raw_ad_file_paths = glob.glob("../../data/raw_data/*h5ad")

In [8]:
for path in raw_ad_file_paths:
    adata = sc.read(path)
    # train scgen
    scgen.SCGEN.setup_anndata(adata, batch_key="tech", labels_key="celltype")
    model = scgen.SCGEN(adata)
    model.save("../../results/scgen_batch_removal.pt", overwrite=True, save_anndata = True)
    model.train(
        max_epochs = 100,
        batch_size = 32,
        early_stopping = True,
        early_stopping_patience = 25,
    )
    
    res = model.batch_removal()
    pcr = scib.me.pcr_comparison(adata_pre = adata, adata_post = res, covariate = "celltype")
    nmi = scib.me.nmi(adata = res, group1 = "tech",  group2 = "celltype")
    ari = scib.me.ari(adata = res, group1 = "tech",  group2 = "celltype")

    int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", "raw", "0", "0", "0",
                         "scgen", "pcr", pcr]
    int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch", "raw", "0", "0", "0",
                         "scgen", "nmi", nmi]
    int_metrics.loc[len(int_metrics.index)] = ["path", "human_pancreas_norm_complexBatch",  "raw", "0", "0", "0",
                                       "scgen", "pcr", ari]         
    print(int_metrics)
    int_metrics.to_csv("scgen.csv")

/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/X' was written without encoding metadata.
  d[k] = read_elem(f[k])
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/layers' was written without encoding metadata.
  d[k] = read_elem(f[k])
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:92: OldFormatWarning: Element '/layers/counts' was written without encoding metadata.
  return {k: read_elem(v) for k, v in elem.items()}
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/tech' was written without encoding metadata.
  categories = read_elem(categories_dset)
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:587: OldFormatWarning: Element '/obs/t

Epoch 5/100:   4%|▏    | 4/100 [05:22<2:08:02, 80.02s/it, loss=7.7e+03, v_num=1]

/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup       
Variance contribution increased after integration!
Setting PCR comparison score to 0.
      ad                           dataset              c_type n_cell  \
0   path  human_pancreas_norm_complexBatch  activated_stellate      0   
1   path  human_pancreas_norm_complexBatch  activated_stellate      0   
2   path  human_pancreas_norm_complexBatch  activated_stellate      0   
3   path  human_pancreas_norm_complexBatch                beta      0   
4   path  human_pancreas_norm_complexBatch                beta      0   
5   path  human_pancreas_norm_complexBatch                beta      0   
6   path  human_pancreas_norm_complexBatch              acinar      0   
7   path  human_pancreas_norm_complexBatch              acinar      0   
8   path  human_pancreas_norm_complexBatch              acinar      0   
9   path  human_pancreas_norm_complexBatch               alpha      0   
10  path  human_p

/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/layers' was written without encoding metadata.
  d[k] = read_elem(f[k])
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/batchname' was written without encoding metadata.
  categories = read_elem(categories_dset)
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:587: OldFormatWarning: Element '/obs/batchname' was written without encoding metadata.
  read_elem(dataset), categories, ordered=ordered
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/batchname_all' was written without encoding metadata.
  categories = read_elem(categories_dset)
/Users/miaoxinran/anaconda3/envs/scvi-env2/lib/python3.9/site-packages/anndata/

KeyError: 'tech not found in adata.obs.'